# Next_Word_prediction

### By: ZaraPhCode

Here is the code that uses a file containing texts with different topics, and design an LSTM Neural Network to predict the next word in a sequence. 

I got some help from this Source especially with the dataset:

[link text](https://www.geeksforgeeks.org/next-word-prediction-with-deep-learning-in-nlp/)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

Here we define a function to open a text file and remove unneccessary characters. Then we transform this list of words into a categorial list in which any word has got an specific label.

In [2]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

file_path = '/content/sample_data/1661-0.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

Designing the LSTM network:

In [3]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

fitting the network:

In [4]:
# Train the model
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
3312/3312 [==============================] - 98s 27ms/step - loss: 6.4967 - accuracy: 0.0642
Epoch 2/50
3312/3312 [==============================] - 45s 14ms/step - loss: 5.8042 - accuracy: 0.1033
Epoch 3/50
3312/3312 [==============================] - 42s 13ms/step - loss: 5.4110 - accuracy: 0.1301
Epoch 4/50
3312/3312 [==============================] - 43s 13ms/step - loss: 5.1304 - accuracy: 0.1443
Epoch 5/50
3312/3312 [==============================] - 41s 13ms/step - loss: 4.8792 - accuracy: 0.1542
Epoch 6/50
3312/3312 [==============================] - 42s 13ms/step - loss: 4.6525 - accuracy: 0.1646
Epoch 7/50
3312/3312 [==============================] - 43s 13ms/step - loss: 4.4468 - accuracy: 0.1755
Epoch 8/50
3312/3312 [==============================] - 41s 12ms/step - loss: 4.2569 - accuracy: 0.1884
Epoch 9/50
3312/3312 [==============================] - 41s 13ms/step - loss: 4.0826 - accuracy: 0.2014
Epoch 10/50
3312/3312 [==============================] - 41s 12m

For having better result, 200 epochs and more is recommended.
Lets see how our network perform after training for only 50 epochs:

In [6]:
# Generate next word predictions
seed_text = "Books are good for "
next_words = 3

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)


1/1 [==============================] - 0s 80ms/step
Next predicted words: Books are good for  some days or


Not bad!